<a href="https://colab.research.google.com/github/crossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
import requests
import io

# --- Configurações de Estilo (Mantidas do seu código) ---
FIGSIZE = (12, 8)
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black', 'axes.facecolor': 'black', 'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan', 'xtick.color': 'white', 'ytick.color': 'white',
    'text.color': 'white', 'grid.color': 'gray', 'grid.linestyle': '--',
    'legend.facecolor': 'black', 'legend.edgecolor': 'white', 'figure.titlesize': 20,
    'axes.titlesize': 16, 'axes.labelsize': 14, 'xtick.labelsize': 12, 'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# --- Funções Auxiliares (Ligeiramente modificadas) ---
def mount_google_drive():
    """Monta o Google Drive."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Caminho completo no Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Cria diretório no Drive se não existir."""
    drive_path = get_drive_path(relative_path)
    os.makedirs(drive_path, exist_ok=True)
    return drive_path

def save_fig(fig, filename, drive_folder_path):
    """Salva figura no Drive."""
    filepath = os.path.join(drive_folder_path, filename)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Importante!

def create_figure():
    """Cria figura matplotlib."""
    return plt.figure(figsize=FIGSIZE)  # Usa FIGSIZE

def generate_statistical_summary(df, filename, drive_path):
    """Gera resumo estatístico e salva."""
    summary = df.describe(include='all').to_string()
    filepath = os.path.join(drive_path, filename)
    with open(filepath, 'w') as f:
        f.write(summary)

# --- Função de Plotagem Genérica (Modificada) ---
def generate_plot(df, plot_type, x_col, y_col, title, filename, drive_folder_path, hue=None, limit=6, **kwargs):
    """Função genérica para gerar gráficos, com limite e estilo."""
    try:
        if x_col not in df.columns or (y_col is not None and y_col not in df.columns):
            print(f"Aviso: Colunas '{x_col}' ou '{y_col}' ausentes. Gráfico '{title}' não gerado.")
            return

        fig = create_figure()
        ax = fig.add_subplot(1, 1, 1)

        # Aplica o limite de categorias para o hue *e* para a variável principal (x_col)
        if hue:
            top_hues = df[hue].value_counts().index[:limit]
            data = df[df[hue].isin(top_hues)]
        else:
            data = df

        # Limita também a variável no eixo x (importante para countplot, boxplot, violinplot)
        if plot_type in ('countplot', 'boxplot', 'violinplot'):
            top_x = data[x_col].value_counts().index[:limit]
            data = data[data[x_col].isin(top_x)]


        if plot_type == 'lineplot':
            sns.lineplot(x=x_col, y=y_col, data=data, hue=hue, palette="viridis", ax=ax, **kwargs)
        elif plot_type == 'violinplot':
            sns.violinplot(x=x_col, y=y_col, data=data, hue=hue, palette="viridis", ax=ax, split=(hue is not None), **kwargs)
        elif plot_type == 'kdeplot':
            sns.kdeplot(data=data, x=x_col, hue=hue, palette="viridis", ax=ax, fill=True, **kwargs) # Adicionado fill=True
        elif plot_type == 'stacked_barplot':
            if y_col is None:
                print(f"Aviso: y_col não especificado para stacked bar plot. '{title}' não gerado.")
                plt.close(fig)
                return
            pivot_table = pd.pivot_table(data, values=y_col, index=x_col, columns=hue, aggfunc=np.sum, fill_value=0)
            pivot_table.plot(kind='bar', stacked=True, colormap="viridis", ax=ax, **kwargs)
        elif plot_type == 'countplot':  # Adicionado countplot
            sns.countplot(x=x_col, data=data, hue=hue, palette="viridis", ax=ax, order=data[x_col].value_counts().index) # Mantém a ordem
        elif plot_type == 'boxplot':  # Adicionado boxplot
            sns.boxplot(x=x_col, y=y_col, data=data, hue=hue, palette="viridis", ax=ax, **kwargs)
        elif plot_type == 'scatterplot': # Adicionado scatterplot
             sns.scatterplot(x=x_col, y=y_col, data=data, hue=hue, palette="viridis", ax=ax, **kwargs)
        else:
            print(f"Tipo de gráfico desconhecido: {plot_type}")
            plt.close(fig)
            return

        ax.set_title(title)
        ax.set_xlabel(x_col.replace('_', ' '))  # Melhora a formatação dos rótulos
        if y_col:  # Só define ylabel se y_col existir
            ax.set_ylabel(y_col.replace('_', ' '))

        # Rotaciona rótulos do eixo x se forem muito longos (especialmente útil para countplot)
        if plot_type in ('countplot', 'boxplot', 'violinplot', 'stacked_barplot'):
            plt.xticks(rotation=45, ha="right")

        save_fig(fig, filename, drive_folder_path)

    except Exception as e:
        print(f"Erro ao gerar o gráfico '{title}': {e}")
        plt.close(fig)


# --- Função Principal (Adaptada) ---
if __name__ == "__main__":
    try:
        mount_google_drive()
        graficos_drive_path = ensure_directory_exists_on_drive('data')

        url = "https://docs.google.com/spreadsheets/d/1e8otjghJ3PGvUNnQZevu6bZ0ktjBfWWUXvkRm61_y_8/export?format=csv"
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text))

        # Limpeza e pré-processamento
        df.dropna(axis=1, how='all', inplace=True)
        for col in df.select_dtypes(include=np.number):
            df[col] = df[col].fillna(df[col].median())
        df.columns = df.columns.str.strip()
        # Encoding (se necessário, e com nomes de colunas corretos)
        if 'grupo' in df.columns:
          df['grupo_encoded'] = pd.Categorical(df['grupo']).codes
        if 'feedback_professor' in df.columns:
          df['feedback_encoded'] = pd.Categorical(df['feedback_professor']).codes

        # --- Lista de Gráficos (Adaptada) ---
        plots = [
            ('lineplot', "desempenho_prova", "frequencia", "1. Desempenho vs Frequência", "1_lineplot.png", 'grupo_encoded'),
            ('violinplot', "grupo", "desempenho_prova", "2. Desempenho por Grupo", "2_violinplot.png", 'grupo_encoded'),
            ('kdeplot', "satisfacao", None, "3. KDE da Satisfação", "3_kdeplot.png", 'grupo_encoded'),
            ('stacked_barplot', "grupo", "participacao_atividades", "4. Participação por Grupo", "4_stackedbarplot.png", 'feedback_encoded'),
            ('lineplot', "desempenho_prova", "centralidade_comunicacao", "5. Desempenho vs Centralidade", "5_lineplot.png", 'grupo_encoded'),
            ('violinplot', "feedback_professor", "desempenho_prova", "6. Desempenho por Feedback", "6_violinplot.png", 'feedback_encoded'),
            ('kdeplot', "diversidade_equipe", None, "7. KDE da Diversidade", "7_kdeplot.png", 'grupo_encoded'),
            ('stacked_barplot', "grupo", "desempenho_prova", "8. Desempenho por Grupo (Stacked)", "8_stackedbarplot.png", 'feedback_encoded'),
            ('lineplot', "frequencia", "satisfacao", "9. Frequência vs Satisfação", "9_lineplot.png", 'grupo_encoded'),
            ('violinplot', "grupo", "centralidade_comunicacao", "10. Centralidade por Grupo", "10_violinplot.png", 'grupo_encoded'),
            ('kdeplot', "desempenho_prova", None, "11. KDE do Desempenho", "11_kdeplot.png", 'feedback_encoded'),
            ('stacked_barplot', "feedback_professor", "satisfacao", "12. Satisfação por Feedback", "12_stackedbarplot.png", 'grupo_encoded'),
            ('lineplot', "participacao_atividades", "desempenho_prova", "13. Participação vs Desempenho", "13_lineplot.png", 'grupo_encoded'),
            ('violinplot', "feedback_professor", "satisfacao", "14. Satisfação por Feedback (Violin)", "14_violinplot.png", 'feedback_encoded'),
            ('kdeplot', "questionario_socioemocional", None, "15. KDE do Questionário Socioemocional", "15_kdeplot.png", 'grupo_encoded'),
            ('countplot', "grupo", None, "16. Contagem por Grupo", "16_countplot.png"), # Exemplo de countplot
            ('boxplot', "feedback_professor", "desempenho_prova", "17. Boxplot Desempenho por Feedback", "17_boxplot.png"), # Exemplo de boxplot
            ('scatterplot', "desempenho_prova", "satisfacao", "18. Scatterplot Desempenho vs Satisfação", "18_scatterplot.png", 'grupo_encoded'), # Exemplo de scatterplot

        ]

        for plot_args in plots:
            plot_type, x_col, y_col, title, filename, *hue = plot_args
            hue = hue[0] if hue else None
            generate_plot(df, plot_type, x_col, y_col, title, filename, graficos_drive_path, hue)

        generate_statistical_summary(df, "sumario.txt", graficos_drive_path)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar os dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-7-91e36eff30d6>:83: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data=data, x=x_col, hue=hue, palette="viridis", ax=ax, fill=True, **kwargs) # Adicionado fill=True
<ipython-input-7-91e36eff30d6>:89: FutureWarning: The provided callable <function sum at 0x7f3be13e8a40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(data, values=y_col, index=x_col, columns=hue, aggfunc=np.sum, fill_value=0)


Erro ao gerar o gráfico '7. KDE da Diversidade': The x variable is categorical, but one of ['numeric', 'datetime'] is required


<ipython-input-7-91e36eff30d6>:89: FutureWarning: The provided callable <function sum at 0x7f3be13e8a40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(data, values=y_col, index=x_col, columns=hue, aggfunc=np.sum, fill_value=0)
<ipython-input-7-91e36eff30d6>:89: FutureWarning: The provided callable <function sum at 0x7f3be13e8a40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table = pd.pivot_table(data, values=y_col, index=x_col, columns=hue, aggfunc=np.sum, fill_value=0)
<ipython-input-7-91e36eff30d6>:92: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countp